In [1]:
import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
from tensorflow.keras import layers
import math
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Activation, Flatten, Dense, Reshape
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import os
import pickle
from keras.utils import plot_model
from numpy import array
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.convolutional import MaxPooling1D, MaxPooling2D
import time
from keras.preprocessing.image import ImageDataGenerator
import os, shutil
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
import tensorflow as tf
from tensorflow.keras import optimizers
#from keras.models import Model
#from keras.applications.vgg16 import VGG16
#from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
#from keras.models import Model
from matplotlib import pyplot
from numpy import expand_dims
from keras import backend as K
import keras

Using TensorFlow backend.


In [2]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [3]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [4]:
train_dir = 'Organized_Dataset/Training/Normal'

In [7]:
fnames = [os.path.join(train_dir, fname) for
         fname in os.listdir(train_dir)]

In [8]:
img_path = fnames[3]
img = image.load_img(img_path, target_size=(64, 64))

x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)

In [10]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(64,64,3))

conv_base.trainable=False

In [11]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0     

In [13]:
model = tf.keras.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [14]:
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc',f1_m,precision_m, recall_m])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 15,764,289
Trainable params: 1,049,601
Non-trainable params: 14,714,688
_________________________________________________________________


In [16]:
train_dir = 'Organized_Dataset/Training/'
validation_dir = 'Organized_Dataset/Validation'

In [17]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

val_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=30,
    class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(64, 64),
    batch_size=30,
    class_mode='binary')

Found 3145 images belonging to 2 classes.
Found 416 images belonging to 2 classes.


In [19]:
history = model.fit_generator(
    training_set,
    steps_per_epoch=30,
    epochs=100,
    validation_data=val_set,
    validation_steps=30)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 30 steps, validate for 30 steps
Epoch 1/100
30/30 [==============================] - 15s 489ms/step - loss: 0.6853 - acc: 0.6500 - f1_m: 0.3756 - precision_m: 0.4167 - recall_m: 0.3833 - val_loss: 0.6550 - val_acc: 0.5250 - val_f1_m: 0.3297 - val_precision_m: 0.4833 - val_recall_m: 0.2667
Epoch 2/100
30/30 [==============================] - 10s 335ms/step - loss: 0.6000 - acc: 0.6917 - f1_m: 0.4067 - precision_m: 0.5056 - recall_m: 0.3778 - val_loss: 0.5285 - val_acc: 0.8833 - val_f1_m: 0.9021 - val_precision_m: 0.8861 - val_recall_m: 0.9472
Epoch 3/100
30/30 [==============================] - 10s 326ms/step - loss: 0.5277 - acc: 0.7500 - f1_m: 0.5833 - precision_m: 0.6611 - recall_m: 0.5889 - val_loss: 0.4551 - val_acc: 0.8833 - val_f1_m: 0.9032 - val_precision_m: 0.8639 - val_recall_m: 0.9750
Epoch 4/100
30/30 [==============================] - 11s 3

Epoch 31/100
30/30 [==============================] - 10s 341ms/step - loss: 0.2067 - acc: 0.9167 - f1_m: 0.7552 - precision_m: 0.7722 - recall_m: 0.7583 - val_loss: 0.1457 - val_acc: 0.9583 - val_f1_m: 0.9705 - val_precision_m: 0.9500 - val_recall_m: 1.0000
Epoch 32/100
30/30 [==============================] - 9s 301ms/step - loss: 0.2577 - acc: 0.9000 - f1_m: 0.8086 - precision_m: 0.8333 - recall_m: 0.8083 - val_loss: 0.2003 - val_acc: 0.9000 - val_f1_m: 0.9190 - val_precision_m: 0.8750 - val_recall_m: 1.0000
Epoch 33/100
30/30 [==============================] - 9s 307ms/step - loss: 0.3074 - acc: 0.8583 - f1_m: 0.7416 - precision_m: 0.7528 - recall_m: 0.7694 - val_loss: 0.2003 - val_acc: 0.9000 - val_f1_m: 0.9190 - val_precision_m: 0.8750 - val_recall_m: 1.0000
Epoch 34/100
30/30 [==============================] - 9s 297ms/step - loss: 0.2875 - acc: 0.8750 - f1_m: 0.7441 - precision_m: 0.7778 - recall_m: 0.7472 - val_loss: 0.2039 - val_acc: 0.9000 - val_f1_m: 0.9190 - val_precision_

Epoch 63/100
30/30 [==============================] - 10s 321ms/step - loss: 0.2967 - acc: 0.9333 - f1_m: 0.8356 - precision_m: 0.8667 - recall_m: 0.8333 - val_loss: 0.1275 - val_acc: 0.9500 - val_f1_m: 0.9594 - val_precision_m: 0.9333 - val_recall_m: 1.0000
Epoch 64/100
30/30 [==============================] - 9s 296ms/step - loss: 0.2335 - acc: 0.9083 - f1_m: 0.8241 - precision_m: 0.8389 - recall_m: 0.8417 - val_loss: 0.1370 - val_acc: 0.9500 - val_f1_m: 0.9594 - val_precision_m: 0.9333 - val_recall_m: 1.0000
Epoch 65/100
30/30 [==============================] - 9s 294ms/step - loss: 0.2065 - acc: 0.9333 - f1_m: 0.8511 - precision_m: 0.8667 - recall_m: 0.8556 - val_loss: 0.1925 - val_acc: 0.8917 - val_f1_m: 0.9143 - val_precision_m: 0.8667 - val_recall_m: 1.0000
Epoch 66/100
30/30 [==============================] - 9s 298ms/step - loss: 0.3083 - acc: 0.8417 - f1_m: 0.5752 - precision_m: 0.5861 - recall_m: 0.6056 - val_loss: 0.1277 - val_acc: 0.9500 - val_f1_m: 0.9594 - val_precision_

Epoch 95/100
30/30 [==============================] - 9s 301ms/step - loss: 0.3077 - acc: 0.8500 - f1_m: 0.7633 - precision_m: 0.8056 - recall_m: 0.7833 - val_loss: 0.1263 - val_acc: 0.9500 - val_f1_m: 0.9594 - val_precision_m: 0.9333 - val_recall_m: 1.0000
Epoch 96/100
30/30 [==============================] - 9s 289ms/step - loss: 0.1956 - acc: 0.9167 - f1_m: 0.8378 - precision_m: 0.8500 - recall_m: 0.8556 - val_loss: 0.1067 - val_acc: 0.9583 - val_f1_m: 0.9724 - val_precision_m: 0.9611 - val_recall_m: 0.9917
Epoch 97/100
30/30 [==============================] - 9s 298ms/step - loss: 0.2486 - acc: 0.9167 - f1_m: 0.7819 - precision_m: 0.8250 - recall_m: 0.7778 - val_loss: 0.1255 - val_acc: 0.9500 - val_f1_m: 0.9594 - val_precision_m: 0.9333 - val_recall_m: 1.0000
Epoch 98/100
30/30 [==============================] - 9s 292ms/step - loss: 0.3552 - acc: 0.8833 - f1_m: 0.7378 - precision_m: 0.7833 - recall_m: 0.7444 - val_loss: 0.1656 - val_acc: 0.9250 - val_f1_m: 0.9368 - val_precision_m

In [22]:
#For Covid vs Other


classifier.compile(optimizer='adam', loss='binary_crossentropy',
                   metrics=['accuracy'])

train_imagedata = ImageDataGenerator(rescale=1. / 255, shear_range=0.2,
        zoom_range=0.2, horizontal_flip=True)
test_imagedata = ImageDataGenerator(rescale=1. / 255)
training_set = \
    train_imagedata.flow_from_directory('Organized_Dataset/Covid-vs-Others-Test/Training'
        , target_size=(64, 64), batch_size=4, class_mode='binary')
val_set = \
    test_imagedata.flow_from_directory('Organized_Dataset/Covid-vs-Others-Test/Validation'
        , target_size=(64, 64), batch_size=4, class_mode='binary')

Found 1795 images belonging to 2 classes.
Found 257 images belonging to 2 classes.


In [23]:
model2 = tf.keras.Sequential()
model2.add(conv_base)
model2.add(layers.Flatten())
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(512, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc',f1_m,precision_m, recall_m])
history2 = model2.fit_generator(
    training_set,
    steps_per_epoch=30,
    epochs=100,
    validation_data=val_set,
    validation_steps=30)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 30 steps, validate for 30 steps
Epoch 1/100
30/30 [==============================] - 13s 442ms/step - loss: 0.3317 - acc: 0.9083 - f1_m: 0.9444 - precision_m: 0.9167 - recall_m: 0.9917 - val_loss: 0.4533 - val_acc: 0.8500 - val_f1_m: 0.9079 - val_precision_m: 0.8500 - val_recall_m: 1.0000
Epoch 2/100
30/30 [==============================] - 10s 335ms/step - loss: 0.1706 - acc: 0.9583 - f1_m: 0.9762 - precision_m: 0.9583 - recall_m: 1.0000 - val_loss: 0.4806 - val_acc: 0.8500 - val_f1_m: 0.9079 - val_precision_m: 0.8500 - val_recall_m: 1.0000
Epoch 3/100
30/30 [==============================] - 9s 284ms/step - loss: 0.2266 - acc: 0.9412 - f1_m: 0.9651 - precision_m: 0.9417 - recall_m: 1.0000 - val_loss: 0.4356 - val_acc: 0.8500 - val_f1_m: 0.9079 - val_precision_m: 0.8500 - val_recall_m: 1.0000
Epoch 4/100
30/30 [==============================] - 10s 326ms/step - loss: 0.2656 - acc: 0.9167 - f1_m: 0.9524 - precision_m: 0.9167 -

Epoch 32/100
30/30 [==============================] - 8s 270ms/step - loss: 0.2526 - acc: 0.9664 - f1_m: 0.9794 - precision_m: 0.9667 - recall_m: 1.0000 - val_loss: 0.1564 - val_acc: 0.9333 - val_f1_m: 0.9584 - val_precision_m: 0.9306 - val_recall_m: 1.0000
Epoch 33/100
30/30 [==============================] - 8s 277ms/step - loss: 0.1164 - acc: 0.9750 - f1_m: 0.9819 - precision_m: 0.9806 - recall_m: 0.9889 - val_loss: 0.2151 - val_acc: 0.9083 - val_f1_m: 0.9425 - val_precision_m: 0.9056 - val_recall_m: 1.0000
Epoch 34/100
30/30 [==============================] - 9s 301ms/step - loss: 0.1986 - acc: 0.9167 - f1_m: 0.9413 - precision_m: 0.9250 - recall_m: 0.9722 - val_loss: 0.1158 - val_acc: 0.9250 - val_f1_m: 0.9537 - val_precision_m: 0.9306 - val_recall_m: 0.9917
Epoch 35/100
30/30 [==============================] - 8s 276ms/step - loss: 0.1462 - acc: 0.9667 - f1_m: 0.9752 - precision_m: 0.9806 - recall_m: 0.9778 - val_loss: 0.1877 - val_acc: 0.9250 - val_f1_m: 0.9537 - val_precision_m

Epoch 64/100
30/30 [==============================] - 8s 262ms/step - loss: 0.0321 - acc: 0.9917 - f1_m: 0.9952 - precision_m: 1.0000 - recall_m: 0.9917 - val_loss: 0.2865 - val_acc: 0.9083 - val_f1_m: 0.9422 - val_precision_m: 0.9028 - val_recall_m: 1.0000
Epoch 65/100
30/30 [==============================] - 8s 269ms/step - loss: 0.0700 - acc: 0.9667 - f1_m: 0.9810 - precision_m: 0.9833 - recall_m: 0.9833 - val_loss: 0.3830 - val_acc: 0.8917 - val_f1_m: 0.9311 - val_precision_m: 0.8861 - val_recall_m: 1.0000
Epoch 66/100
30/30 [==============================] - 8s 283ms/step - loss: 0.1735 - acc: 0.9500 - f1_m: 0.9714 - precision_m: 0.9500 - recall_m: 1.0000 - val_loss: 0.1481 - val_acc: 0.9333 - val_f1_m: 0.9584 - val_precision_m: 0.9306 - val_recall_m: 1.0000
Epoch 67/100
30/30 [==============================] - 8s 274ms/step - loss: 0.0360 - acc: 0.9750 - f1_m: 0.9857 - precision_m: 0.9833 - recall_m: 0.9917 - val_loss: 0.1416 - val_acc: 0.9333 - val_f1_m: 0.9584 - val_precision_m

Epoch 96/100
30/30 [==============================] - 8s 278ms/step - loss: 0.1045 - acc: 0.9750 - f1_m: 0.9819 - precision_m: 0.9889 - recall_m: 0.9806 - val_loss: 0.1616 - val_acc: 0.9250 - val_f1_m: 0.9537 - val_precision_m: 0.9222 - val_recall_m: 1.0000
Epoch 97/100
30/30 [==============================] - 8s 262ms/step - loss: 0.1036 - acc: 0.9667 - f1_m: 0.9810 - precision_m: 0.9833 - recall_m: 0.9833 - val_loss: 0.2345 - val_acc: 0.9167 - val_f1_m: 0.9489 - val_precision_m: 0.9139 - val_recall_m: 1.0000
Epoch 98/100
30/30 [==============================] - 8s 264ms/step - loss: 0.1315 - acc: 0.9583 - f1_m: 0.9746 - precision_m: 0.9722 - recall_m: 0.9806 - val_loss: 0.1887 - val_acc: 0.9167 - val_f1_m: 0.9489 - val_precision_m: 0.9139 - val_recall_m: 1.0000
Epoch 99/100
30/30 [==============================] - 8s 265ms/step - loss: 0.1050 - acc: 0.9833 - f1_m: 0.9886 - precision_m: 0.9806 - recall_m: 1.0000 - val_loss: 0.1586 - val_acc: 0.9333 - val_f1_m: 0.9581 - val_precision_m